# 05-06 트러블 슈팅

## imshow assertion 실패

In [ ]:
import cv2

cap = cv2.VideoCapture('./data/no_such_video.mp4')  # 없는 파일

while True:
    ret, frame = cap.read()
    # ret이 False인데도 frame을 출력하려고 하면 오류
    cv2.imshow('Video', frame)   # 여기서 Assertion failed 발생
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.12.0) C:\opencv\opencv-4.12.0\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


## MSMF 문제

In [ ]:
import cv2

# MSMF로 열 경우 문제 발생 가능
cap1 = cv2.VideoCapture('./data/input.mp4', cv2.CAP_MSMF)

# FFMPEG으로 강제 지정 → 보통 안정적
cap2 = cv2.VideoCapture('./data/input.mp4', cv2.CAP_FFMPEG)

## FPS 문제

In [4]:
import cv2

cap = cv2.VideoCapture('./data/1718887518_sample_640x360.mp4')  # 가변 프레임 레이트 예제

print('Reported Frame Count:', cap.get(cv2.CAP_PROP_FRAME_COUNT))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
    pos_msec  = cap.get(cv2.CAP_PROP_POS_MSEC)
    print(f'Frame={pos_frame}, Time={pos_msec}ms')

cap.release()

Reported Frame Count: 400.0
Frame=1.0, Time=0.0ms
Frame=2.0, Time=33.36666666666667ms
Frame=3.0, Time=66.73333333333333ms
Frame=4.0, Time=100.10000000000001ms
Frame=5.0, Time=133.46666666666667ms
Frame=6.0, Time=166.83333333333334ms
Frame=7.0, Time=200.20000000000002ms
Frame=8.0, Time=233.56666666666666ms
Frame=9.0, Time=266.93333333333334ms
Frame=10.0, Time=300.3ms
Frame=11.0, Time=333.6666666666667ms
Frame=12.0, Time=367.03333333333336ms
Frame=13.0, Time=400.40000000000003ms
Frame=14.0, Time=433.7666666666667ms
Frame=15.0, Time=467.1333333333333ms
Frame=16.0, Time=500.50000000000006ms
Frame=17.0, Time=533.8666666666667ms
Frame=18.0, Time=567.2333333333333ms
Frame=19.0, Time=600.6ms
Frame=20.0, Time=633.9666666666667ms
Frame=21.0, Time=667.3333333333334ms
Frame=22.0, Time=700.7ms
Frame=23.0, Time=734.0666666666667ms
Frame=24.0, Time=767.4333333333334ms
Frame=25.0, Time=800.8000000000001ms
Frame=26.0, Time=834.1666666666667ms
Frame=27.0, Time=867.5333333333334ms
Frame=28.0, Time=900.90

## 저장한 동영상이 재생안되거나 뒤틀림

In [ ]:
import cv2
import numpy as np

# 640x480으로 저장기로 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("wrong.mp4", fourcc, 30, (640, 480))

# 실제 프레임은 320x240 → 불일치
frame = np.full((240, 320, 3), (0, 0, 255), dtype=np.uint8)

for _ in range(30):
    out.write(frame)  # 뒤틀림/재생 불가
out.release()

## 한글 문제

In [10]:
import cv2
import numpy as np

# 일반 imread는 실패 가능 (예: "C:/데이터/고양이.jpg")
path = './data/고양이01.jpg'

# 실패함
# img = cv2.imread(path, cv2.IMREAD_COLOR)

# 안전한 방식
data = np.fromfile(path, dtype=np.uint8)
img = cv2.imdecode(data, cv2.IMREAD_COLOR)

cv2.imshow('Unicode Path Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
!pip install ffmpeg-python


   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 2/2 [ffmpeg-python]



In [13]:
import ffmpeg
import numpy as np
import cv2

path = "./data/샘플영상.mp4"

# ffmpeg으로 첫 프레임만 읽기 예시
out, _ = (
    ffmpeg
    .input(path)
    .output("pipe:", format="rawvideo", pix_fmt="bgr24", vframes=1)
    .run(capture_stdout=True)
)

frame = np.frombuffer(out, np.uint8).reshape([-1, 640, 3])  # (height는 실제 해상도에 맞게 조정)
cv2.imshow("Frame", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다